In [165]:
import pandas as pd
import numpy as np
import requests
import json
pd.set_option('display.max_columns', None)


In [19]:
with open("tmdb.json", "r") as fp:
    key = json.load(fp)

api_key = key["api_key"]

def get_movie_details(
    TMdb_id: int,
    use_IMdb: bool = False,
    IMdb_id: str = ""
):
    """Récupère les détails d'un film par son ID TMDB."""
    if use_IMdb:
        base_url = "https://api.themoviedb.org/3/find/"
        url = f"{base_url}{IMdb_id}?api_key={api_key}&external_source=imdb_id&language=fr"
    else:
        base_url = "https://api.themoviedb.org/3/movie/"
        url = f"{base_url}{movie_id}?api_key={api_key}&language=fr"
    response = requests.get(url)
    return response.json()

movie_id = 872585
movie_details = get_movie_details(TMdb_id=movie_id, use_IMdb=True, IMdb_id="tt15398776")
import pprint
pprint.pprint(movie_details)
print(movie_details['title'])
print(movie_details['overview'])
print(movie_details['spoken_languages'])


{'movie_results': [{'adult': False,
                    'backdrop_path': '/rLb2cwF3Pazuxaj0sRXQ037tGI1.jpg',
                    'genre_ids': [18, 36],
                    'id': 872585,
                    'media_type': 'movie',
                    'original_language': 'en',
                    'original_title': 'Oppenheimer',
                    'overview': 'En 1942, convaincus que l’Allemagne nazie est '
                                'en train de développer une arme nucléaire, '
                                'les États-Unis initient, dans le plus grand '
                                'secret, le "Projet Manhattan" destiné à '
                                'mettre au point la première bombe atomique de '
                                'l’histoire. Pour piloter ce dispositif, le '
                                'gouvernement engage J. Robert Oppenheimer, '
                                'brillant physicien, qui sera bientôt surnommé '
                                '"le pèr

KeyError: 'title'

In [21]:

def get_urls() -> dict:
    return {
        "genres" : "https://api.themoviedb.org/3/genre/movie/list"
    }


params = {
    "api_key": api_key,
    'include_adult': False,
    "language": "en",
}

pd.rea

response = requests.get(get_urls()["genres"], params)
data = response.json()
data
new_dic = {
    g['id']: g["name"] for
    g in data["genres"]
}
new_dic

{28: 'Action',
 12: 'Adventure',
 16: 'Animation',
 35: 'Comedy',
 80: 'Crime',
 99: 'Documentary',
 18: 'Drama',
 10751: 'Family',
 14: 'Fantasy',
 36: 'History',
 27: 'Horror',
 10402: 'Music',
 9648: 'Mystery',
 10749: 'Romance',
 878: 'Science Fiction',
 10770: 'TV Movie',
 53: 'Thriller',
 10752: 'War',
 37: 'Western'}

In [143]:
from datetime import datetime
date = datetime.strftime(datetime.now(), '%Y-%m-%d')
base_url = "https://api.themoviedb.org/3/discover/movie"

params = {
    "api_key": api_key,
    'include_adult': False,
    "language": "fr",
    "sort_by": "primary_release_date.desc",
    "primary_release_date.gte" : "2023-05-01",
    "primary_release_date.lte" : date,
    "page": 1
}
all_movies_df = pd.DataFrame()

# plutot que de faire par page, donné uniquement une date
# tmdb_id_ = []
for page in range(1, 501):
    # tmdb_id = {}
    params['page'] = page
    response = requests.get(base_url, params=params)
    data = response.json()
    page_df = pd.DataFrame(data['results'])
    all_movies_df = pd.concat([all_movies_df, page_df], ignore_index=True)
# tmdb_id_
# tmdb_id
# 1183514
all_movies_df

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,None,"[9648, 80]",1201495,ru,Очень синяя борода,,0.000,/6WKIp7OSItzeXVcNJGRvX28SB2P.jpg,2023-11-06,Очень синяя борода,False,0.0,0
1,False,/bZjiHrONMyx9USjo6w2Eb8PLBIv.jpg,[],1201420,xx,Sunset,,0.000,/dmInwd2dxZ2GikhYVggbNF2fcu6.jpg,2023-11-06,Sunset,False,0.0,0
2,False,None,"[28, 9648, 53]",1201418,zh,斗破阴阳宅,,0.000,/8VAn2OFDC6H3Bu3dCBRLmtgrpAk.jpg,2023-11-06,斗破阴阳宅,False,0.0,0
3,False,/cAkpEUNqDouVYDfqA0fy9FqNPzb.jpg,[10749],1201331,mr,Fake Marriage,,0.000,/4TSRBk4TmbnIqhVkE8QR5mFtOQJ.jpg,2023-11-06,Fake Marriage,False,0.0,0
4,False,None,[18],1201224,en,Klepto,,1.400,/79EH9zFwHviY643zbJXAbZg6U1y.jpg,2023-11-06,Klepto,False,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,False,/bCfbhcczkGbK1fUZaL8lWZNHRcL.jpg,[18],669362,fr,Salamandra,,16.861,/y6FrmvwrqRQ5a5gky4LLmPkGNYC.jpg,2023-07-27,Salamandra,False,10.0,1
9996,False,/51rUSdGJRcH4kBKfc8vZET6Mook.jpg,[18],663036,es,Alma,,1.037,/zwtkjjD0rFn5ciqWS4ivUktiNbR.jpg,2023-07-27,Alma,False,0.0,0
9997,False,/4FmATqXPQbTVtOhhIbMFNF3F8xe.jpg,"[18, 80, 9648]",1197998,en,Silent as the Grave,,3.026,/gDeXCxGd0GXrl3xiGXP9caPinhV.jpg,2023-07-26,Silent as the Grave,False,0.0,0
9998,False,/iwVwdGc2VLxU3IQAnNY5TC3c7di.jpg,[35],1186660,en,Settling Down,,1.400,/vurdii19lHpQcAKVHMwJW99aHgm.jpg,2023-07-26,Settling Down,False,0.0,0


In [144]:
from datetime import datetime
date = datetime.strftime(datetime.now(), '%Y-%m-%d')
base_url = "https://api.themoviedb.org/3/discover/movie"


last_day = all_movies_df.release_date.iloc[-1]

params = {
    "api_key": api_key,
    'include_adult': False,
    "language": "fr",
    "sort_by": "primary_release_date.desc",
    "primary_release_date.gte" : "2023-05-01",
    "primary_release_date.lte" : last_day,
    "page": 1
}
response = requests.get(base_url, params=params)
data = response.json()
total_pages = data['total_pages']

# plutot que de faire par page, donné uniquement une date
for page in range(1, total_pages+1):
    params['page'] = page
    response = requests.get(base_url, params=params)
    data = response.json()
    page_df = pd.DataFrame(data["results"])
    all_movies_df = pd.concat([all_movies_df, page_df], ignore_index=True)


# all_movies
# 7523
all_movies_df

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,None,"[9648, 80]",1201495,ru,Очень синяя борода,,0.000,/6WKIp7OSItzeXVcNJGRvX28SB2P.jpg,2023-11-06,Очень синяя борода,False,0.0,0
1,False,/bZjiHrONMyx9USjo6w2Eb8PLBIv.jpg,[],1201420,xx,Sunset,,0.000,/dmInwd2dxZ2GikhYVggbNF2fcu6.jpg,2023-11-06,Sunset,False,0.0,0
2,False,None,"[28, 9648, 53]",1201418,zh,斗破阴阳宅,,0.000,/8VAn2OFDC6H3Bu3dCBRLmtgrpAk.jpg,2023-11-06,斗破阴阳宅,False,0.0,0
3,False,/cAkpEUNqDouVYDfqA0fy9FqNPzb.jpg,[10749],1201331,mr,Fake Marriage,,0.000,/4TSRBk4TmbnIqhVkE8QR5mFtOQJ.jpg,2023-11-06,Fake Marriage,False,0.0,0
4,False,None,[18],1201224,en,Klepto,,1.400,/79EH9zFwHviY643zbJXAbZg6U1y.jpg,2023-11-06,Klepto,False,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17518,False,/xSRviKmGHDv6NDlhbrxJJhlP4Fg.jpg,"[10749, 18, 14]",960197,en,Man of Arcadia,,0.600,/eHdgpKqWYA6IL1j5FYwMI9AA7PU.jpg,2023-05-01,Man of Arcadia,False,0.0,0
17519,False,None,[],894592,it,Upside Down,,1.536,/r1E8DgjtaSvnJjJfYDjY4mwIVfQ.jpg,2023-05-01,Upside Down,False,0.0,0
17520,False,/2t092J6JOS4exjhkmX01X8Vsink.jpg,[18],852865,en,Giving Birth to a Butterfly,,4.178,/Ge2xVAtvGctwHg3tDMvSwen1zG.jpg,2023-05-01,Giving Birth to a Butterfly,False,4.0,2
17521,False,None,[27],772567,es,Justine,,0.780,/6v2O5FN9gvtYFQub5XNqkIPA2Zh.jpg,2023-05-01,Justine,False,0.0,0


In [147]:
all_movies_df.drop_duplicates(subset=["id"], keep='first', inplace=True)

In [152]:
list_id_tmdb = all_movies_df.id.to_list()

In [154]:
small = list_id_tmdb[:5]

In [155]:
small

[1201495, 1201420, 1201418, 1201331, 1201224]

In [166]:

with open("tmdb.json", "r") as fp:
    key = json.load(fp)

api_key = key["api_key"]

def get_movie_details(
    TMdb_id: int,
    use_IMdb: bool = False,
    IMdb_id: str = ""
):
    """Récupère les détails d'un film par son ID TMDB."""
    if use_IMdb:
        base_url = "https://api.themoviedb.org/3/find/"
        url = f"{base_url}{IMdb_id}?api_key={api_key}&external_source=imdb_id&language=fr"
    else:
        base_url = "https://api.themoviedb.org/3/movie/"
        url = f"{base_url}{TMdb_id}?api_key={api_key}&language=fr"
    response = requests.get(url)
    return [response.json()]

movie_id = 872585
pandas = pd.DataFrame()
for id in small:
    print(id)
    movie_details = get_movie_details(TMdb_id=id, use_IMdb=False, IMdb_id="tt15398776")
    print(movie_details)
    movies = pd.DataFrame(movie_details)
    pandas = pd.concat([pandas, movies], ignore_index=True)

pandas


1201495
[{'adult': False, 'backdrop_path': None, 'belongs_to_collection': None, 'budget': 0, 'genres': [{'id': 9648, 'name': 'Mystère'}, {'id': 80, 'name': 'Crime'}], 'homepage': '', 'id': 1201495, 'imdb_id': None, 'original_language': 'ru', 'original_title': 'Очень синяя борода', 'overview': '', 'popularity': 0.0, 'poster_path': '/6WKIp7OSItzeXVcNJGRvX28SB2P.jpg', 'production_companies': [], 'production_countries': [], 'release_date': '2023-11-06', 'revenue': 0, 'runtime': 0, 'spoken_languages': [], 'status': 'Released', 'tagline': '', 'title': 'Очень синяя борода', 'video': False, 'vote_average': 0.0, 'vote_count': 0}]
1201420
[{'adult': False, 'backdrop_path': '/bZjiHrONMyx9USjo6w2Eb8PLBIv.jpg', 'belongs_to_collection': None, 'budget': 0, 'genres': [], 'homepage': '', 'id': 1201420, 'imdb_id': None, 'original_language': 'xx', 'original_title': 'Sunset', 'overview': '', 'popularity': 0.0, 'poster_path': '/dmInwd2dxZ2GikhYVggbNF2fcu6.jpg', 'production_companies': [], 'production_count

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,None,None,0,"[{'id': 9648, 'name': 'Mystère'}, {'id': 80, '...",,1201495,None,ru,Очень синяя борода,,0.0,/6WKIp7OSItzeXVcNJGRvX28SB2P.jpg,[],[],2023-11-06,0,0,[],Released,,Очень синяя борода,False,0.0,0
1,False,/bZjiHrONMyx9USjo6w2Eb8PLBIv.jpg,None,0,[],,1201420,None,xx,Sunset,,0.0,/dmInwd2dxZ2GikhYVggbNF2fcu6.jpg,[],[],2023-11-06,0,3,"[{'english_name': 'No Language', 'iso_639_1': ...",Released,,Sunset,False,0.0,0
2,False,None,None,0,"[{'id': 28, 'name': 'Action'}, {'id': 9648, 'n...",,1201418,None,zh,斗破阴阳宅,,0.0,/8VAn2OFDC6H3Bu3dCBRLmtgrpAk.jpg,[],[],2023-11-06,0,0,[],Released,,斗破阴阳宅,False,0.0,0
3,False,/cAkpEUNqDouVYDfqA0fy9FqNPzb.jpg,None,0,"[{'id': 10749, 'name': 'Romance'}]",,1201331,None,mr,Fake Marriage,,0.0,/4TSRBk4TmbnIqhVkE8QR5mFtOQJ.jpg,[],[],2023-11-06,0,0,"[{'english_name': 'Marathi', 'iso_639_1': 'mr'...",Released,,Fake Marriage,False,0.0,0
4,False,None,None,0,"[{'id': 18, 'name': 'Drame'}]",,1201224,None,en,Klepto,,1.4,/79EH9zFwHviY643zbJXAbZg6U1y.jpg,"[{'id': 159028, 'logo_path': None, 'name': 'TF...",[],2023-11-06,0,4,[],Released,,Klepto,False,0.0,0


In [ ]:
# # ici je recupere le dernier jour.
# last_day = all_movies_df.release_date.iloc[-1]
# # test = datetime.strptime(last_day, "%Y-%m-%d") + timedelta(days=1)
# # str_date = datetime.strftime(test,  "%Y-%m-%d")
# # str_date
# print(last_day)